In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import (accuracy_score,
                                classification_report,
                                confusion_matrix,
                                roc_auc_score, 
                                fbeta_score,
                                make_scorer, 
                                )

import warnings
warnings.filterwarnings('ignore')

In [2]:
abt = pd.read_csv('analytical_base_table.csv')

In [3]:
y = abt.bad_flag
X = abt.drop('bad_flag', axis=1)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=1234,
    stratify=abt.bad_flag
)

In [5]:
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
scale_pos_weight = neg / pos
print("scale_pos_weight =", scale_pos_weight)   

scale_pos_weight = 8.952095808383234


In [12]:

f2_scorer = make_scorer(fbeta_score, beta=2)

xgb_model = XGBClassifier(
    random_state=123,
    scale_pos_weight=scale_pos_weight,
    eval_metric='aucpr',
    use_label_encoder=False,
)


hyperparameters_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2]
}

grid = GridSearchCV(
    xgb_model,
    hyperparameters_xgb,
    cv=10,
    scoring=f2_scorer,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

best_xgb = grid.best_estimator_
print("Best params:", grid.best_params_)
print("Best score:", grid.best_score_)

y_pred = best_xgb.predict(X_test)
y_pred_proba = best_xgb.predict_proba(X_test)[:, 1]
print(f"AUC: {roc_auc_score(y_test, y_pred_proba):.3f}\n")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred), "\n")

print("Classification Report:")
print(classification_report(y_test, y_pred))


f2 = fbeta_score(y_test, y_pred, beta=2)
print(f"F2-score: {f2:.3f}")


Fitting 10 folds for each of 36 candidates, totalling 360 fits
Best params: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 200}
Best score: 0.4882122866784478
AUC: 0.791

Confusion Matrix:
[[591 158]
 [ 30  53]] 

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.79      0.86       749
           1       0.25      0.64      0.36        83

    accuracy                           0.77       832
   macro avg       0.60      0.71      0.61       832
weighted avg       0.88      0.77      0.81       832

F2-score: 0.488
